In [ ]:
pip install pmdarima

In [ ]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot 

In [ ]:
df = pd.read_csv('./data/MaunaLoaDailyTemps.csv',index_col='DATE',parse_dates=True)
df = df.dropna()
df.shape
df.head()

In [ ]:
df['AvgTemp'].plot(figsize=(9,5))

In [ ]:
# check statinary or not by dicky fuller test.
# the above plot is loking that the dataset is stationary but chec it by dicky fuller test
# to assure.
from statsmodels.tsa.stattools import adfuller

# .)The blw p_value is used if it is lower than 0.5 so it is stationary if not hight value then
# itsnot stationary.
dftest = adfuller(df['AvgTemp'],autolag='AIC')
print("1. ADF : ",dftest[0])
print("2. P_value  : ",dftest[1])
print("3. Num od Lags : ",dftest[2])
print("4. Num od observations used for ADF regression and critical values calculation : ",dftest[3])
print("5. Critical values : ") 
for key, val in dftest[4].items():
    print("\t",key, ": ",val)
#            .)p value is 8 so its not statinary .         

In [ ]:

# arima(p,d,q)
# p is lags of auto reg d is differnce to to make data statinary and q is lags of moving 
# avg.

# figure out order for arima .
# .)The blw auto_arima will automatyically see and tke out good values of p,d, and q .
from pmdarima import auto_arima 
# .)Trace true will print every step.
stepwise_fit = auto_arima(df['AvgTemp'],trace=True,suppress_warnings=True)
stepwise_fit.summary()

In [ ]:
# .)The above auto-arima give followig reslt .
# Best model:  ARIMA(1,0,5)(0,0,0)[0] intercept
# Total fit time: 40.135 seconds

In [ ]:
train = df.iloc[:-30]
test = df.iloc[-30:]
print(train.shape,test.shape)

In [ ]:
import statsmodels.api as sm
model=sm.tsa.arima.ARIMA(train['AvgTemp'],order=(1,0,5))
model=model.fit()
model.summary()

In [ ]:
# make pred on test .
start = len(train)
end = len(train)+len(test)-1
pred = model.predict(start=start,end=end,type='levels')


In [ ]:
pred.index = df.index[start:end+1]
print(pred)

In [ ]:
pred.plot(legend=True)
test['AvgTemp'].plot(legend=True)

In [ ]:

from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(pred,test['AvgTemp']))
print(rmse)

In [ ]:
# .)Training model again and pred future values.
model2=sm.tsa.arima.ARIMA(df['AvgTemp'],order=(1,0,5))
model2=model2.fit()
df.tail()

In [ ]:
# for future dates.
index_future_dates=pd.date_range(start='2018-12-30',end='2019-01-29')
#print(index_future_dates)
pred=model2.predict(start=len(df),end=len(df)+30,typ='levels').rename('ARIMA Predictions')
#print(comp_pred)
pred.index=index_future_dates
print(pred)

In [ ]:
pred.plot(figsize=(12,5),legend=True)